In [ ]:
%pip install -q "openai>=1.40.0" "httpx>=0.27.2" "httpcore>=1.0.5" "pandas>=2.2.2" "tqdm>=4.66.4"

In [ ]:
import os, getpass, json, time, platform
from importlib.metadata import version, PackageNotFoundError

def ver(pkg):
    try:
        return version(pkg)
    except PackageNotFoundError:
        return None

# ==== Configure DeepSeek ====
MODEL    = os.getenv("DEEPSEEK_MODEL", "Deepseek-VL-7B-Chat")
BASE_URL = os.getenv("DEEPSEEK_BASE_URL", "https://api.deepseek.com")

api_key = os.getenv("DEEPSEEK_API_KEY") or os.getenv("API_KEY")
if not api_key:
    api_key = getpass.getpass("Paste your DeepSeek API key: ")
os.environ["DEEPSEEK_API_KEY"] = api_key

save_dir  = "./results_DeepseekVL7BChat"
meta_path = os.path.join(save_dir, "meta.json")

meta = {
    "model": MODEL,
    "base_url": BASE_URL,
    "python": platform.python_version(),
    "time_utc": time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()),
    "openai_pkg": ver("openai"),
    "httpx": ver("httpx"),
}

os.makedirs(save_dir, exist_ok=True)
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print("Meta saved ->", meta_path)
print("MODEL:", MODEL, "| BASE_URL:", BASE_URL)
print("DEEPSEEK_API_KEY set:", bool(os.environ.get("DEEPSEEK_API_KEY")))


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=os.environ["DEEPSEEK_API_KEY"], base_url=BASE_URL)
print("Client ready.")

In [ ]:
# Vision-language sanity check
# You can provide either:
# 1) a local image filepath via IMAGE_PATH (it will be base64-encoded), or
# 2) an image URL via IMAGE_URL (publicly accessible).
import os, base64

image_url = os.getenv("IMAGE_URL")
image_path = os.getenv("IMAGE_PATH")

content_parts = [{"type": "text", "text": "Describe what's in this image in one sentence."}]

if image_path and os.path.exists(image_path):
    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    content_parts.append({
        "type": "image_url",
        "image_url": {"url": f"data:image/png;base64,{b64}"}
    })
elif image_url:
    content_parts.append({
        "type": "image_url",
        "image_url": {"url": image_url}
    })
else:
    print("No image provided via IMAGE_PATH or IMAGE_URL; running a text-only prompt instead.")

msgs = [
    {"role": "system", "content": "You are a helpful multimodal assistant."},
    {"role": "user", "content": content_parts},
]

resp = client.chat.completions.create(
    model=MODEL,
    messages=msgs,
    temperature=0.2,
    max_tokens=200,
)
print(resp.choices[0].message.content)